In [101]:
import os
import numpy as np
import pandas as pd
import glob

from collections import Counter
from nltk import word_tokenize
import nltk

### embed data string to list

In [102]:
## load_data
with open("../data/all.review.vec.txt", 'r', encoding="UTF8") as f:
    review_vec = f.readlines()
all_review_vec = review_vec[1:]

In [103]:
embed_vec = np.zeros((56050, 100))
key_word = []
for i in range(len(all_review_vec)):
    key_word.append(all_review_vec[i].split()[0])
    for j in range(100):
        embed_vec[i, j] = float(all_review_vec[i].split()[j+1])

In [104]:
embed_vec_dict = {}
for i in range(len(key_word)):
    embed_vec_dict[key_word[i]] = embed_vec[i, :]
len(embed_vec_dict)

56050

### split token

In [105]:
positive_files = glob.glob("../data/train/positive/*")
negative_files = glob.glob("../data/train/negative/*")
total_files = positive_files + negative_files

In [106]:
token_list = []
token_counts = []
for file_root in total_files:
    file = open(file_root, 'r', encoding="UTF8")
    reviews = file.readline()
    token = word_tokenize(reviews)
    token_list += token
    token_counts.append(len(token))
print('min_num : ', np.min(token_counts) ,',', 'max_num : ', np.max(token_counts),',', 'token_list :', len(token_list))

min_num :  9 , max_num :  3817 , token_list : 375720


### delete token which is not in review_vec

In [107]:
for i in reversed(range(len(token_list))):
    if token_list[i] not in key_word:
        del token_list[i]
print('token_list :', len(token_list))

token_list : 369990


### most word token within 10k

In [116]:
count = Counter(token_list)
print('unique_token :', len(count))

unique_token : 19388


In [117]:
most_count = count.most_common(10000)
print('unique_token :', len(most_count))

unique_token : 10000


In [131]:
most_token = []

for token, count in most_count:
    most_token.append(token)

### most 10k token embedding preprocessing

In [166]:
final_embed = pd.DataFrame({"token": [], "embedding_vector": []})

for i in range(len(most_count)):
    final_embed.loc[i] = [most_token[i], embed_vec_dict[most_token[i]].tolist()]
final_embed.head()

c:\users\wonsang\anaconda3\envs\ca1\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


,token,embedding_vector
0,the,"[0.077042, -0.138293, 0.045446, 0.001055, -0.0..."
1,",","[0.077455, -0.196322, -0.036069, -0.026043, -0..."
2,.,"[0.10809, -0.2071, -0.018723, -2.7e-05, -0.143..."
3,and,"[0.096765, -0.117544, 0.010411, 0.008122, -0.1..."
4,of,"[0.188291, -0.204709, 0.006558, -0.132994, -0...."


In [189]:
with open('../data/10k_embed.txt', 'w') as f:
    for i, vector in enumerate(final_embed["embedding_vector"]):
        tmp = [str(embed) for embed in vector]
        output = str(i) + ' '+ ' '.join(tmp) + '\n'
        f.write(output)

## list

In [209]:
train_negative_list = glob.glob("../data/train/negative/*")
train_positive_list = glob.glob("../data/train/positive/*")
print( 'negative_list: ', len(train_negative_list),' ' +  'positive_list :', len(train_positive_list))
train_total_list = train_negative_list + train_positive_list

negative_list:  1000  positive_list : 1000


In [210]:
tags_word2idx = {}
for i in range(len(most_token)):
    tags_word2idx[most_token[i]] = i

### train

In [195]:
train = np.zeros((2000, 3774)) # 3817
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [206]:
for i, file_root in enumerate(train_total_list):
    file = open(file_root, 'r', encoding="UTF8")
    reviews = file.readline()
    if file_root in train_negative_list:
        label = 0
    else:
        label = 1
    token = word_tokenize(reviews)
    for k in reversed(range(len(token))):
        if (token[k] not in key_word):
            del token[k]

    for j in range(len(token)):
        if token[j] in most_token:
            train[i, j] = tags_word2idx[token[j]]
        else:
            train[i, j] = 0
        train[i, 3773] = label

../data/train/negative\1909
../data/train/negative\1916
../data/train/negative\1917
../data/train/negative\1918
../data/train/negative\192
../data/train/negative\1920
../data/train/negative\1922
../data/train/negative\1924
../data/train/negative\1926
../data/train/negative\1927
../data/train/negative\1928
../data/train/negative\1929


In [207]:
np.save("../data/train_data", train)

### test

In [211]:
test_list = glob.glob("../data/test/*")

negative_list:  0  positive_list : 0


In [213]:
test = np.zeros((2000, 3774)) # 3817

In [214]:
for i, file_root in enumerate(test_list):
    file = open(file_root, 'r', encoding="UTF8")
    reviews = file.readline()
    token = word_tokenize(reviews)
    for k in reversed(range(len(token))):
        if (token[k] not in key_word):
            del token[k]

    for j in range(len(token)):
        if token[j] in most_token:
            test[i, j] = tags_word2idx[token[j]]
        else:
            test[i, j] = 0

In [215]:
np.save("../data/test_data", test)